### Web Search Tool

This tool lets the LLM answer questions using its own knowledge by default,  
and only calls the Tavily web search tool when up-to-date or web-based  
information is required.


In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
import getpass
import os

if not os.environ.get("GROQ_API_KEY"):
  os.environ["GROQ_API_KEY"] = getpass.getpass("Enter API key for Groq: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("llama-3.1-8b-instant", model_provider="groq")

In [3]:
from langchain_tavily import TavilySearch

tool = TavilySearch(
    max_results=5,
    topic="general",
    # include_answer=False,
    # include_raw_content=False,
    # include_images=False,
    # include_image_descriptions=False,
    # search_depth="basic",
    # time_range="day",
    # include_domains=None,
    # exclude_domains=None
)

In [4]:
tool.invoke({"query": "Leo Messi last 3 opponent team when playing for barcelona"})

{'query': 'Leo Messi last 3 opponent team when playing for barcelona',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://talksport.com/football/2040978/lionel-messi-toughest-opponent-manchester-city-pablo-maffeo-barcelona/',
   'title': "Lionel Messi's toughest opponent spent three years at Man City and ...",
   'content': 'Messi named current Mallorca right-back Pablo Maffeo. Formerly of Manchester City, Maffeo spent three years on the books at the Etihad between 2015 and 2018.',
   'score': 0.59880555,
   'raw_content': None},
  {'url': 'https://www.givemesport.com/lionel-messi-named-strongest-opponent-team-played/',
   'title': 'Lionel Messi Named Strongest Team He Ever Played - GiveMeSport',
   'content': 'Messi only faced the Spanish national team three times in his career, and never in a major tournament. All three meetings between Argentina and',
   'score': 0.5809471,
   'raw_content': None},
  {'url': 'https://www.facebook.com/10Messit

In [5]:
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langchain_groq import ChatGroq

# ----------------------------
# Tavily search tool
# ----------------------------

tavily = TavilySearch(max_results=5, topic="general")

@tool
def web_search(query: str):
    """Use this tool ONLY when the question requires recent, real-time,
    or web-based information (e.g., current events, breaking news, live stats).
    If the question can be answered from your own knowledge, do NOT call this tool."""
    return tavily.invoke({"query": query})


# ----------------------------
# Init Groq LLaMA model
# ----------------------------

llm = ChatGroq(model="llama-3.1-8b-instant")

# Bind only Tavily tool
llm_with_tools = llm.bind_tools([web_search])


In [6]:
resp = llm_with_tools.invoke("Hello who are you?")

In [7]:
resp

AIMessage(content='I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 298, 'total_tokens': 321, 'completion_time': 0.023950361, 'prompt_time': 0.01802833, 'queue_time': 0.053178309, 'total_time': 0.041978691}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_8804b970d6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--d4998b7d-3e49-4842-a8dc-bebbba573ee1-0', usage_metadata={'input_tokens': 298, 'output_tokens': 23, 'total_tokens': 321})

In [8]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Text box for user query
input_box = widgets.Text(
    value='',
    placeholder='Type your question...',
    description='You:',
    disabled=False
)

# Button to submit query
button = widgets.Button(description="Ask")

output = widgets.Output()

def on_button_click(b):
    with output:
        clear_output(wait=True)
        user_input = input_box.value.strip()
        if not user_input:
            print("Please enter a query.")
            return
        print("You:", user_input)
        try:
            response = llm_with_tools.invoke(user_input)
            print("Assistant:", response)
        except Exception as e:
            print("Error:", e)

button.on_click(on_button_click)

display(input_box, button, output)


Text(value='', description='You:', placeholder='Type your question...')

Button(description='Ask', style=ButtonStyle())

Output()